In [2]:
import cv2
import os
import requests
import numpy as np
from PIL import Image
from transformers import BlipProcessor, BlipForConditionalGeneration, AutoProcessor, GitVisionModel, AutoModel
import scipy # for saving to wav file

# set seed for reproducability
np.random.seed(45)

ModuleNotFoundError: No module named 'cv2'

## Capture frames from a video input

OpenCV, frame-by-frame

In [3]:
video_path = 'data/video.mp4'
cap = cv2.VideoCapture(video_path)

output_directory = 'output_frames'
os.makedirs(output_directory, exist_ok=True)

fps = cap.get(cv2.CAP_PROP_FPS)
interval_seconds = 5
interval_frames = int(fps * interval_seconds)
count = 1

if not cap.isOpened():
    print("Error opening video file")
else:
    while cap.isOpened():
        # Capture frame-by-frame
        ret, frame = cap.read()

        if not ret:
            break

        if count % interval_frames == 0:
            output_path = os.path.join(output_directory, f"frame_{count}.jpg")
            cv2.imwrite(output_path, frame)
            print(f"Saved frame {count}")
        count += 1
    cap.release()

NameError: name 'cv2' is not defined

In [4]:
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-large")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-large")

NameError: name 'BlipProcessor' is not defined

In [49]:
if os.path.exists(output_directory):
    for filename in os.listdir(output_directory):
        
        if filename.endswith(".jpg"):
            print(f"Processing {filename}")
            raw_image = Image.open(os.path.join(output_directory, filename), "r").convert('RGB')
            
            print("Unconditional blip")
            # unconditional image captioning
            inputs = processor(raw_image, return_tensors="pt")
            out = model.generate(**inputs)
            print(processor.decode(out[0], skip_special_tokens=True))

        else:
            print(f"Skipping {filename}")
        
else:
    print(f"Directory '{directory}' does not exist.")


#img_url = 'https://storage.googleapis.com/sfr-vision-language-research/BLIP/demo.jpg'
#raw_image = Image.open(requests.get(img_url, stream=True).raw).convert('RGB')

Skipping .ipynb_checkpoints
Processing frame_149.jpg
Conditional blip
the mood of the image is very interesting and it is very beautiful
Unconditional blip
trees line the road in the middle of a forest with sun shining through the trees
Processing frame_298.jpg
Conditional blip
the mood of the image is very blurry and the city lights are bright
Unconditional blip
night time view of a city with lights from a hill
Processing frame_447.jpg
Conditional blip
the mood of the image is very beautiful and the birds are sitting on the gate
Unconditional blip
there are birds sitting on a fence in the sun


## Music generation with MusicGen

In [23]:
from transformers import AutoProcessor, MusicgenForConditionalGeneration

music_processor = AutoProcessor.from_pretrained("facebook/musicgen-small")
model = MusicgenForConditionalGeneration.from_pretrained("facebook/musicgen-small")

In [28]:
# pass in descriptions
inputs = processor(
    text=descriptions,
    padding=True,
    return_tensors="pt",
)
audio_values = model.generate(**inputs, do_sample=True, guidance_scale=3, max_new_tokens=1503)

Listen either using jupyter, or export as wav file

In [32]:
from IPython.display import Audio

sampling_rate = model.config.audio_encoder.sampling_rate
Audio(audio_values[0].numpy(), rate=sampling_rate)
#Audio(audio_values[1].numpy(), rate=sampling_rate)

In [ ]:
sampling_rate = model.config.audio_encoder.sampling_rate
scipy.io.wavfile.write("musicgen_out.wav", rate=sampling_rate, data=audio_values[0, 0].numpy())

## Video generation

In [50]:
import torch
from diffusers import DiffusionPipeline, DPMSolverMultistepScheduler
from diffusers.utils import export_to_video

# load pipeline
pipe = DiffusionPipeline.from_pretrained("damo-vilab/text-to-video-ms-1.7b", torch_dtype=torch.float16, variant="fp16")
pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)

# optimize for GPU memory
pipe.enable_model_cpu_offload()
pipe.enable_vae_slicing()

# generate
prompt = "Spiderman is surfing. Darth Vader is also surfing and following Spiderman"
video_frames = pipe(prompt, num_inference_steps=25, num_frames=200).frames

# convent to video
video_path = export_to_video(video_frames)
print(video_path)

Fetching 12 files:   0%|          | 0/12 [00:00<?, ?it/s]

KeyboardInterrupt: 